In [58]:
import json

import ijson
import pandas as pd

import pymongo
from pymongo import MongoClient
from tqdm.notebook import tqdm

In [60]:
index = {}

with open('SARC2/comments.txt', 'w') as txt:
    with open('SARC2/comments.json', 'r') as f:
        for i, (id_str, comment) in enumerate(ijson.kvitems(f, '')):
            index[id_str] = i
            txt.write(f"{comment['text']}\n")

In [62]:
index['c07nkao']

72

In [64]:
len(index)

12704751

In [67]:
def get_nth_line(n):
    fp = open('SARC2/comments.txt', 'r')
    for i, line in enumerate(fp):
        if i == n:
            break
    fp.close()
    return line

get_nth_line(12704751)

'You think Trump is a conservative?\n'

In [83]:
def load_dataset(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    ret = []
    for line in tqdm(lines):
        line = line.split('|')
        comments = line[1].split(' ')
        labels = line[2].strip().split(' ')
        
        comment1 = comments[0]
        comment2 = comments[1]
        label1 = int(labels[0])
        label2 = int(labels[1])
        
        c1_idx = index[comment1]
        c2_idx = index[comment2]
        
        # comment1 = get_nth_line(c1_idx)
        # comment2 = get_nth_line(c2_idx)
        
        ret.append([c1_idx, label1])
        ret.append([c2_idx, label2])
        
    return pd.DataFrame(ret, columns=["idx", "label"])

sarc = load_dataset("SARC2/test-balanced.csv")

  0%|          | 0/32333 [00:00<?, ?it/s]

In [99]:
def read_comments(idxs):
    idxs = set(idxs)
    idxs = list(sorted(idxs))
    
    ret = {}
    
    fp = open('SARC2/comments.txt', 'r')
    for i, line in tqdm(enumerate(fp)):
        if idxs and idxs[0] == i:
            ret[idxs[0]] = line.strip()
            idxs.pop(0)
    
    return ret

big_index = read_comments(sarc["idx"])

0it [00:00, ?it/s]

In [103]:
sarc['text'] = sarc['idx'].map(lambda x: big_index[x])
sarc

,idx,label,text
0,72,1,"Yes, cuz tax cuts will help those w/o jobs!"
1,1403415,0,If cutting taxes fails... cut taxes harder.
2,89,1,Chalk it up to the ever-increasing cost of fre...
3,1403435,0,"We're about to finally get affordable housing,..."
4,125,1,oh wow I am so surprised I never saw this coming
...,...,...,...
64661,12704317,0,THEYRE SO CUTE
64662,12704350,0,I'm very curious to see if they'll actually fo...
64663,1403227,1,That's Bartkowski!
64664,1403251,1,"But according the Chafetz, they are rich so th..."


In [108]:
sarc.drop(columns=['idx']).to_csv('SARC2/sarc_processed.csv')